# Problem:
## Given a cell in [Arbor](https://arbor.readthedocs.io/en/latest/index.html#) that has been set up How does one create a PyNN population composed of this cell?

### 1.Create a cell
See `PyNN_arbor_settingup_a_cell.ipynb` for description of the steps.
#### 1.1. First the arbor-morphology is created

In [2]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology

In [3]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))
pynn_morph = NeuroMLMorphology(neuroml_morph)

In [4]:
import arbor
import re

In [5]:
# methods already defined in PyNN_neuroml_morphology_IN_Arbor.ipynb
def create_arbor_tree(pynn_nml_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(pynn_nml_morph.segments):
        append_arbor_tree(tree, nml_seg)
    return tree
    
def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(arbor.mnpos,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))

def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return 2
    elif re.search("dend", nml_seg.name, re.IGNORECASE):
        return 3
    else:
        return 5

In [6]:
tree = create_arbor_tree(pynn_morph)

In [7]:
#Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
def create_region_definitions(pynn_morph):
    dict_defs = {}
    for i, nml_seg in enumerate(pynn_morph.segments):
        dict_defs.update({nml_seg.name: "(tag "+ str(get_swc_tag(nml_seg))+ ")"})
    dict_defs.update({"everywhere": "(all)"})
    return dict_defs

In [8]:
#dict_defs => {'soma': '(tag 1)', 'dendrite': '(tag 3)'}
tree_labels = arbor.label_dict(create_region_definitions(pynn_morph))

#### 1.2. Create cell

In [9]:
# Cell builders need to refer to regions and locations on a cell morphology.
cell = arbor.cable_cell(tree, tree_labels)

#### 1.2. Set cell properties

In [10]:
# Set cell-wide properties that will be applied by default to # the entire cell.
#cell.set_properties(Vm=-70, cm=0.02, rL=30, tempK=30+273.5)
cell.set_properties(cm=1.0, rL=500.)

#### 1.3. Set ion properties (for channel mechanisms)

In [11]:
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)

#### 1.4. Set channel mechanisms

In [12]:
leaky_chnl = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})

In [13]:
#na_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120})
# na={"conductance_density": uniform('soma', 0.120), "e_rev": 50.0}
#na_chnl = arbor.mechanism("hh", {"ek": -77., "gkbar": 0.036})
# kdr={"conductance_density": uniform('soma', 0.036), "e_rev": -77.0}
hh_na_k_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120, "ek": -77., "gkbar": 0.036, "gl": 0.})

In [14]:
cell.paint("soma", hh_na_k_chnl)
cell.paint("everywhere", leaky_chnl)

In [15]:
#cell.set_properties(Vm=-70, cm=0.02, rL=30, tempK=30+273.5)
cell.set_properties(Vm=-70)